In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_12월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(1011113, 2) Index(['split_str', 'org_idx'], dtype='object')
1011113


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 1011113/1011113 [20:23:25<00:00, 13.77it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['벌써', '주화', '요약', '시작', '어피', '치가', '거', '김시연', '후드', '모자']
['지나다', '나타나다', '따다', '쓸다', '않다', '웃기다', '찍다', '가다', '기다리다', '넘다']
['같다', '귀엽다', '다행하다', '빨갛다', '마르다', '맛있다', '귀엽다', '힘들다', '아니다', '놀래다']
['의', '네', '부터', '아', '아', '이', '고', '고', '이', '만']


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/12월/나무문화_12월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

164434


,명사,명사빈도
0,것,218814
1,수,135510
2,그,113356
3,이,106728
4,사람,75068
...,...,...
164429,뒹구르,1
164430,누런옷,1
164431,친벗,1
164432,만화루,1


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/12월/나무문화_12월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1895


,동사,동사빈도
0,하다,958195
1,되다,196873
2,보다,107417
3,않다,87989
4,되어다,82191
...,...,...
1890,부시럭대다,1
1891,오그라뜨리다,1
1892,사리물다,1
1893,볶아치다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/12월/나무문화_12월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1871


,형용사,형용사빈도
0,있다,437238
1,없다,81164
2,좋다,68732
3,같다,66326
4,이다,62011
...,...,...
1866,민둥하다,1
1867,잔득하다,1
1868,서슴없다,1
1869,새하얘지다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/12월/나무문화_12월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

394


,조사,조사빈도
0,을,784282
1,의,630524
2,이,577483
3,에,560480
4,를,385337
...,...,...
389,로구려,1
390,에게다,1
391,에게서처럼,1
392,에다가는,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_12월_단어종합.txt", index=False, encoding='UTF-8')

In [11]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,흑흑 벌써 1년이 지났다니ㅠㅠ 주화의 2022년 요약 시작 ㅎㅅㅎ 2022 01 어...,0,"[흑, 흑, 벌써, 1년, 이, 지났다니, ㅠㅠ, 주화, 의, 2022년, 요약, ...","[(흑, Adverb), (흑, Adverb), (벌써, Noun), (1년, Nu..."
1,웃겨서 찍음,0,"[웃겨서, 찍음]","[(웃기다, Verb), (찍다, Verb)]"
2,그리고 김시연때뭉네 새해부터 부산감 광안리도 갔는데 횡단보도 기다리는 강아쥐 넘 귀...,0,"[그리고, 김시연, 때뭉, 네, 새해, 부터, 부산, 감, 광안, 리도, 갔는데, ...","[(그리고, Conjunction), (김시연, Noun), (때뭉, Noun), ..."
3,모범 강아쥐 그리고 바로 담날 최성빈이 경주오겠다고 놀아달라고 놀아달라고 해서 놀아줌ㅋ,0,"[모범, 강, 아, 쥐, 그리고, 바로, 담날, 최성빈, 이, 경주, 오겠다고, 놀...","[(모범, Noun), (강, Noun), (아, Josa), (쥐, Noun), ..."
4,녹차라떼 거하게 쏟을 뻔 한거 최성빈이 잡아서 다행히 저만큼만 쏟음,0,"[녹차, 라떼, 거, 하게, 쏟을, 뻔, 한, 거, 최성빈, 이, 잡아서, 다행히,...","[(녹차, Noun), (라떼, Noun), (거, Noun), (하다, Verb)..."
...,...,...,...,...
1011108,돼지의 모습이 신기한지 우리 아이들 이곳에서 한참을 봤네요 ㅎㅎㅎ,20247,"[돼지, 의, 모습, 이, 신기한지, 우리, 아이, 들, 이, 곳, 에서, 한참, ...","[(돼지, Noun), (의, Josa), (모습, Noun), (이, Josa),..."
1011109,남문 주변에는 닭 말 개 등 동물도 가까이서 볼 수 있어요 마을내에는 전통 민박에서...,20247,"[남문, 주변, 에는, 닭, 말, 개, 등, 동물, 도, 가까이, 서, 볼, 수, ...","[(남문, Noun), (주변, Noun), (에는, Josa), (닭, Noun)..."
1011110,문화유산과 공존하며 살아가는 마을 주민들의 삶을 느껴볼 수도 있다고 해요,20247,"[문화유산, 과, 공존, 하며, 살아가는, 마을, 주민, 들, 의, 삶, 을, 느껴...","[(문화유산, Noun), (과, Josa), (공존, Noun), (하다, Ver..."
1011111,제주 성읍민속마을 맛집 성읍치킨튀에서 맛있게 먹고 마을투어 해보는 것을 추천해요 옛...,20247,"[제주, 성, 읍민, 속, 마을, 맛집, 성, 읍, 치킨, 튀에서, 맛있게, 먹고,...","[(제주, Noun), (성, Modifier), (읍민, Noun), (속, Mo..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_12월_형태소분석.txt", index=False, encoding='UTF-8')